Репозиторий оценки качества моделей методом организаторов [CAFA-evaluator](https://github.com/BioComputingUP/CAFA-evaluator)

Оригинальные файлы из репозитория автора:

- evaluation.py
- graph.py
- parser.py  


evaluate_ours.py - запуск оценки на наших данных, завершился нехваткой памяти в  
размере ~19 Гб.

In [12]:
from pathlib import Path

import numpy as np
import pandas as pd

from graph import Graph
from parser import ia_parser, gt_parser, obo_parser, pred_parser
from evaluation import evaluate_prediction


# Корень проекта.
DIR_ROOT = Path.cwd().parent.parent
# Путь к удаленной директории с ресурсами: данные, модели и т.д.
DIR_REMOTE: Path | None = Path('/home/admin/cafa/resources')

if DIR_REMOTE is not None and DIR_REMOTE.exists():
    DIR_RESOURCE = DIR_REMOTE
else:
    DIR_RESOURCE = DIR_ROOT

I Собираем данные из Information accretion file

In [2]:
# Input -> txt file из строк вида GO:0000003 3.27
# Output -> dict(ключи - GO, значения - веса)

ia_dict = ia_parser(DIR_RESOURCE / 'data/raw/IA.txt')

In [3]:
# проверим, что во время парсинга файла, он считался целиком
with open(DIR_RESOURCE / 'data/raw/IA.txt', "r") as f:
    if len(ia_dict) == len(f.readlines()):
        print('OK')
    else:
        print("Файл считался не целиком")

OK


II Собираем данные из файла с онтологиями генов

In [4]:
# Input obo_parser -> obo file (go-basic.obo)
# Output obo_parser -> dict c ключами dict_keys(
# ['biological_process', 'molecular_function', 'cellular_component']
# )
# для каждого ключа значение - словарь с ключами GO и значениями -
# параметрами этих GO
# {
#  'biological_process':
#     {'GO:0000001': {'name': 'mitochondrion inheritance',
#                     'namespace': 'biological_process',
#                     'def': '"The distribution of mitochondria, including the
#                              mitochondrial genome, into daughter cells after
#                              mitosis or meiosis, mediated by interactions
#                              between mitochondria and the cytoskeleton."
#                              [GOC:mcc, PMID:10873824, PMID:11389764]',
#                     'alt_id': [],
#                     'rel': ['GO:0048308', 'GO:0048311']},
#       'GO:0000002': ... },
#   'molecular_function': ...
#   ...
# }

obo_file = DIR_RESOURCE / 'data/raw/Train/go-basic.obo'

ontologies: list[Graph] = []
# ns: str ('biological_process', 'molecular_function', 'cellular_component')
# terms_dict : словарь с GO и их параметрами для каждой ns
# ia_dict: dict(ключи - GO, значения - веса)
# orphans=True: All terms, also those without parents
for ns, terms_dict in obo_parser(obo_file).items():
    ontologies.append(Graph(ns, terms_dict, ia_dict, orphans=True))

In [12]:
# конвертировать файл tsv в txt, убрать столбец с корневыми онтологиями, чтобы
# привести к формату авторского решения

# код выполнен, файл сохранен, запускать только для нового файла ground truth

# train_terms = pd.read_csv(
#   DIR_RESOURCE / 'data/raw/Train/train_terms.tsv', sep='\t', header=None
# )

# train_terms.drop(columns=[2], inplace=True)
# train_terms = train_terms.iloc[1:]

# train_terms.to_csv(
#   DIR_RESOURCE / 'data/raw/Train/train_terms.txt', sep='\t',index=None
# )

III Собираем данные по белкам, у которых известны онтологии (в нашем случае  
это файл train_terms, test_terms есть только у организаторов)

In [9]:
# Input -> gt_file, ontologies
# Output -> dict(namespace: объект GroundTruth(ids, matrix, namespace))

gt_file = DIR_RESOURCE / 'data/raw/Train/train_terms.txt'

gt = gt_parser(gt_file, ontologies)  # около 4Гб RAM

# Потрачено ~ 6,24 Гб Memory

IV Формируем маccив с пороговыми значения от 0.001 до 0.999

In [9]:
th_step = 0.001

tau_arr = np.arange(th_step, 1, th_step)

V Собираем массив из объектов предсказаний по каждой корневой онтологии

In [10]:
file_name = DIR_RESOURCE / 'data/submission/submission_1000.txt'
prop = 'fill'  # by default 'max', but author choosed fill
max_terms = 1500  # количество онтологий, которое мы выбрали при обучении

# predictions: list[Prediction] - Prediction - the score matrix contains the
# scores given by the predictor for every node of the ontology
predictions = pred_parser(file_name, ontologies, gt, prop, max_terms)

MemoryError: Unable to allocate 19.2 GiB for an array with shape (92210, 27942) and data type float64

VI Оценка предсказаний модели разными метриками

Расшифровка метрик для оценки:

- tau == значение порога, который используется при формировании метрик
'
- cov == Coverage -> number of proteins with at least one term predicted with  
       score >= tau

- pr == precision

- rc == recall

- f == f1_score -> a harmonic mean of the precision and recall

- wcov == Weighted coverage, compute missing and remaining uncertainty terms

- wpr == Weighted precision, compute missing and remaining uncertainty terms

- wrc == Weighted recall, compute missing and remaining uncertainty terms

- wf == Weighted f1_score, compute missing and remaining uncertainty terms

- mi == misinformation --> predicted but not in the ground truth

- ru == remaining uncertainty -> not predicted but in the ground truth

- s == compute np.sqrt(ru**2 + mi**2)

- max_cov == метки для лучших результатов, чтобы сформировать потом df_best

In [ ]:
dfs = []

# оценка предсказаний модели по метрикам ("cov", "pr", "rc", "f", "wcov",
# "wpr", "wrc", "wf", "mi", "ru", "s", 'max_cov')
df_pred = evaluate_prediction(predictions, gt, ontologies, tau_arr)
df_pred['filename'] = file_name
dfs.append(df_pred)

VII Расчет средней и запись в файл результатов оценки предсказаний

In [ ]:
# На выходе получаем 1 файл csv evaluation_all.tsv a table containing the full
# evaluation, i.e. assessment measures for each threshold. This file is used
# as input to generate the plots (see below)

# И отдельный файл для каждой метрики с лучшими строками evaluation_best_{}.tsv

out_folder = DIR_RESOURCE / 'путь до папки для сохранения оценки'

df = pd.concat(dfs)

# Save the dataframe
df = df[df['cov'] > 0].reset_index(drop=True)
df.set_index(['filename', 'ns', 'tau'], inplace=True)

columns = [
        "cov", "pr", "rc", "f", "wcov", "wpr", "wrc", "wf", "mi", "ru", "s"
    ]

df.to_csv(
    '{}/evaluation_all.tsv'.format(out_folder),
    columns=columns,
    float_format="%.5f",
    sep="\t",
)

# Calculate harmonic mean across namespaces for each evaluation metric
for metric, cols in [
    ('f', ['rc', 'pr']), ('wf', ['wrc', 'wpr']), ('s', ['ru', 'mi'])
]:
    if metric in columns:
        index_best = (
            df.groupby(level=['filename', 'ns'])[metric].idxmax()
            if metric in ['f', 'wf']
            else df.groupby(['filename', 'ns'])[metric].idxmin()
            )

        df_best = df.loc[index_best]
        df_best['max_cov'] = (
            df.reset_index('tau').loc[
                [ele[:-1] for ele in index_best]
                ].groupby(level=['filename', 'ns'])['cov'].max()
            )
        df_best.to_csv('{}/evaluation_best_{}.tsv'.format(out_folder, metric),
                       columns=columns + ["max_cov"], float_format="%.5f",
                       sep="\t")